In [3]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

try:
# Create a new model
    PwC = gp.Model ("PwT-seqIP")

# processing times of jobs
    protime = [4, 6, 3, 2, 9, 5, 4, 9]

# weights
    weights = [6, 5, 7, 3, 2, 1, 8, 5]
    
# due dates of jobs
    due = [3,10,9,7,5,0,10,5]
    
# # of machines
    m = 3

# range of jobs and machines
    jobs = range(len(protime))
    machines = range(m)

# Create variables
    y = PwC.addVars(machines, jobs, vtype = GRB.BINARY, name = "assign")
    x = PwC.addVars(machines, jobs, jobs, vtype = GRB.BINARY, name = "prec")
    C = PwC.addVars(jobs, vtype = GRB.CONTINUOUS, name = "completion")
    T = PwC.addVars(jobs, obj = weights, lb = 0, name = "Tardiness")

# Set objective
    PwC.modelSense = GRB.MINIMIZE

# Add constraints
    PwC.addConstrs((gp.quicksum(y[i,j] for i in machines) == 1 for j in jobs), name = "assign2machines")
    PwC.addConstrs((x[i,j,k] + x[i,k,j] <= y[i,j] for i in machines for j in jobs for k in jobs if j != k), name = "sequence-1")
    PwC.addConstrs((x[i,j,k] + x[i,k,j] <= y[i,k] for i in machines for j in jobs for k in jobs if j != k), name = "sequence-2")
    PwC.addConstrs((x[i,j,k] + x[i,k,j] >= y[i,j] + y[i,k] - 1 for i in machines for j in jobs for k in jobs if j != k), name = "sequence-3")
    PwC.addConstrs((x[i,j,j] == 0 for i in machines for j in jobs), name = "zero")
    PwC.addConstrs((x[i,j,l] >= x[i,j,k] + x[i,k,l] - 1 for i in machines for j in jobs for k in jobs for l in jobs if (j != k and k != l and j != l)), name = "no-cycle")
    PwC.addConstrs((C[j] == gp.quicksum(protime[k] * x[i,k,j] for i in machines for k in jobs) + protime[j] for j in jobs), name = "completion")
    PwC.addConstrs((T[j]>= C[j]-due[j] for j in jobs))
    PwC.addConstrs((T[j]>= 0 for j in jobs))
# Optimize model
    PwC.optimize()
    PwC.write("PwT-seqIP.lp")

# print optimal solutions
    for v in PwC.getVars ():
        print ('%s %g' % (v.varName , v.x))
    objective = PwC.getObjective()
    print(objective.getValue())

# print optimal sequence of scheduling
    print('An optimal sequence is: ')
    for i in machines:
        print('M%g : ' % i, end = '')
        for n in jobs:
            for j in jobs:
                if (y[i,j].X == 1 and sum(x[i,k,j].X for k in jobs) == n):
                    print('%g --> ' % j, end = '')
                    continue

except gp.GurobiError as e:
    print ('Error code' + str (e. errno ) + ': ' + str(e))
except AttributeError :
    print ('Encountered an attribute error')

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1568 rows, 232 columns and 4976 nonzeros
Model fingerprint: 0x477a8382
Variable types: 16 continuous, 216 integer (216 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 292.0000000
Presolve removed 296 rows and 36 columns
Presolve time: 0.03s
Presolved: 1272 rows, 196 columns, 3976 nonzeros
Variable types: 0 continuous, 196 integer (192 binary)

Root relaxation: objective 3.900000e+01, 16 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   39.00000    0   10  292.00000 